In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 14:15:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame
from pyspark import SparkFiles

# URL of the CSV file
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Distribute the file across all workers
spark.sparkContext.addFile(url)

# Use SparkFiles.get to access the file on each worker node
file_path = SparkFiles.get("home_sales_revised.csv")

# Read the CSV file into a DataFrame
df_home = spark.read.csv(file_path, header=True, inferSchema=True)

# Create a temporary view of the DataFrame
df_home.createOrReplaceTempView("home_sales")

# Query the temporary view and show the results
home_df = spark.sql("SELECT * FROM home_sales LIMIT 5")
home_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

three = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
"""

# Execute the SQL query
spark.sql(three).show()



+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
four = """
SELECT 
    date_built, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 
  AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query
spark.sql(four).show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

five = """
SELECT 
    date_built, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(five).show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

# SQL query to calculate the average price of homes per view rating
six = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query
spark.sql(six).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.1701192855834961 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# SQL query to calculate the average price of homes per view rating
nine = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query
spark.sql(nine).show()

print("--- %s seconds ---" % (time.time() - start_time))

# Run the same query on uncached data (for comparison)

# Step 4a: Uncache the table to ensure it's not cached before running this query
spark.catalog.uncacheTable("home_sales")

# Step 4b: Measure runtime for the uncached query
start_time_uncached = time.time()

# Execute the query again on uncached data
avg_price_per_view_result_uncached = spark.sql(nine)

# Show the result
avg_price_per_view_result_uncached.show()

# Step 4c: Calculate and print the runtime for the uncached query
print("--- Uncached query runtime: %s seconds ---" % (time.time() - start_time_uncached))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.09535098075866699 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Uncached query runtime: 0.1066901683807373 s

In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df_home.write.partitionBy("date_built").parquet("home_sales", mode='overwrite')


In [13]:
# 11. Read the formatted parquet data.
df_home_p = spark.read.parquet('home_sales')

In [14]:
# 12. Create a temporary table for the parquet data.
df_home.createOrReplaceTempView("home_sales")

In [15]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

nine = """
SELECT 
    view, 
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query
spark.sql(nine).show()

print("--- %s seconds ---" % (time.time() - start_time))

# Cache the DataFrame to optimize for repeated queries
df_home.cache()

# Measure runtime for the second query (with caching)
start_time = time.time()

# Run the same SQL query again using the cached DataFrame
spark.sql(nine).show()

# Measure runtime with caching
print("--- Query Runtime with caching: %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.0997929573059082 seconds ---
+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- Query Runtime with caching: 0.153741121292114

In [16]:
# 14. Uncache the home_sales temporary table.
df_home_p.unpersist()

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [20]:
# 15. Check if the home_sales is no longer cached
# Uncache the DataFrame
df_home.unpersist()

# Check if the DataFrame is still cached after unpersist
print(f"Is df_home cached after unpersist? {df_home.is_cached}")

# Check if the home_sales temporary table is cached
is_cached_table = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' table cached? {is_cached_table}")

# Stop the Spark session
spark.stop()

Is df_home cached after unpersist? False
Is 'home_sales' table cached? False
